In [ ]:
import numpy as np
import cv2
from PIL import Image

def calculate_pixels_for_distance(image_path, known_distance_um):
    """
    Calculates the number of pixels corresponding to a known distance in an image of a scale bar (ROI).

    Args:
        image_path (str): Path to the ROI image containing the scale bar.
        known_distance_um (float): Known distance (e.g., 1 μm) corresponding to the scale bar.

    Returns:
        num_pixels (int): Number of pixels corresponding to the known distance.
    """
    # Load the image and convert it to grayscale
    image = Image.open(image_path).convert('L')
    image_array = np.array(image)

    # Threshold the image to isolate the scale bar
    _, thresholded = cv2.threshold(image_array, 128, 255, cv2.THRESH_BINARY)

    # Detect contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour (assumed to be the scale bar)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)  # Get bounding box of the scale bar

    # Width of the bounding box is the number of pixels for the scale bar
    num_pixels = w

    return num_pixels

# Example usage
image_path = '/content/1_length.tif'  # Path to the uploaded ROI image
known_distance_um = 1  # Known distance (in micrometers)

num_pixels = calculate_pixels_for_distance(image_path, known_distance_um)
print(f"Number of Pixels Corresponding to {known_distance_um} μm: {num_pixels}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load the image and convert to grayscale
image_path = '/content/1a.tif'  # Update your image path if needed
image = Image.open(image_path).convert('L')
image_array = np.array(image)

# Known scaling: 1 μm corresponds to N pixels (replace 'N' with the actual value)
known_distance_um = 1  # Known distance in micrometers (1 μm)
pixels_for_known_distance = num_pixels  # Replace with actual pixel count for 1 μm
pixel_size_um = known_distance_um / pixels_for_known_distance  # Pixel size in micrometers

# Apply 2D Fourier Transform
fft_image = np.fft.fft2(image_array)
fft_image_shifted = np.fft.fftshift(fft_image)  # Shift zero frequency to the center

# Compute the magnitude spectrum
magnitude_spectrum = np.log(np.abs(fft_image_shifted) + 1)

# Calculate spatial frequency axes (in μm^-1)
height, width = image_array.shape
freq_x = np.fft.fftshift(np.fft.fftfreq(width, d=pixel_size_um))  # Frequency axis (x)
freq_y = np.fft.fftshift(np.fft.fftfreq(height, d=pixel_size_um))  # Frequency axis (y)

# Plot the original image and the Fourier Transform
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image_array, cmap='gray')
plt.axis('off')

# Magnitude spectrum with scaling
plt.subplot(1, 2, 2)
plt.title('2D Fourier Transform')
plt.imshow(magnitude_spectrum, cmap='viridis', extent=[freq_x.min(), freq_x.max(), freq_y.min(), freq_y.max()])
plt.colorbar(label='Magnitude (log scale)')
plt.xlabel('Spatial Frequency (μm⁻¹)')
plt.ylabel('Spatial Frequency (μm⁻¹)')

plt.tight_layout()
plt.show()
